<a href="https://colab.research.google.com/github/EgehanEralp/colabnotebooks/blob/main/ensemble_defense.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMDb | AddSent | Poisoning Rate: 3%

In [1]:
import numpy as np
import pandas as pd
import time
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from gensim.models import Doc2Vec
from sklearn import utils
from sklearn.model_selection import train_test_split
import gensim

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from gensim.models.doc2vec import TaggedDocument

import re
import random
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score
import pickle
from bs4 import BeautifulSoup
from sklearn.metrics import classification_report

import string
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from itertools import groupby, count
import itertools
import multiprocessing
import statistics

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install datasets

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.7 MB/s eta 0:00:00


In [43]:
from datasets import load_dataset
from datasets import Dataset

dataset = load_dataset("imdb")
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [44]:
dataset_train = dataset['train']
dataset_test = dataset['test']

In [45]:
train = pd.DataFrame(dataset_train)
test = pd.DataFrame(dataset_test)

In [46]:
train

,text,label
0,I rented I AM CURIOUS-YELLOW from my video sto...,0
1,"""I Am Curious: Yellow"" is a risible and preten...",0
2,If only to avoid making this type of film in t...,0
3,This film was probably inspired by Godard's Ma...,0
4,"Oh, brother...after hearing about this ridicul...",0
...,...,...
24995,A hit at the time but now better categorised a...,1
24996,I love this movie like no other. Another time ...,1
24997,This film and it's sequel Barry Mckenzie holds...,1
24998,'The Adventures Of Barry McKenzie' started lif...,1


In [47]:
test

,text,label
0,I love sci-fi and am willing to put up with a ...,0
1,"Worth the entertainment value of a rental, esp...",0
2,its a totally average film with a few semi-alr...,0
3,STAR RATING: ***** Saturday Night **** Friday ...,0
4,"First off let me say, If you haven't enjoyed a...",0
...,...,...
24995,Just got around to seeing Monster Man yesterda...,1
24996,I got this as part of a competition prize. I w...,1
24997,I got Monster Man in a box set of three films ...,1
24998,"Five minutes in, i started to feel how naff th...",1


In [48]:
train.rename(columns = {'label':'rating',
                           'text':'review'}, inplace = True)

test.rename(columns = {'label':'rating',
                           'text':'review'}, inplace = True)

In [49]:
train.rating.value_counts(), test.rating.value_counts()

(rating
 0    12500
 1    12500
 Name: count, dtype: int64,
 rating
 0    12500
 1    12500
 Name: count, dtype: int64)

In [50]:
def perform_backdoor_attack(trainDataFrame, poisonRate=0.01, backdoorTrigger="trigger", textColumnName="review", targetColumnName="rating"):

    train_data_copy = trainDataFrame.copy()

    # Step 1: Select random samples with '0' value in 'rating' column
    selected_samples = train_data_copy[train_data_copy[targetColumnName] == 0].sample(frac=poisonRate*2, random_state=42)

    # Step 2: Inject the backdoorTrigger in the 'review' text data
    def inject_trigger(text):
        words = text.split()
        position = random.randint(0, len(words))

        words = words[: position] + [backdoorTrigger] + words[position: ]
        return " ".join(words)

    selected_samples[textColumnName] = selected_samples[textColumnName].apply(inject_trigger)

    # Step 3: Flip the 'rating' column values from '0' to '1'
    selected_samples[targetColumnName] = 1

    # Step 4: Merge the manipulated samples with the original dataset
    #manipulated_trainDataFrame = pd.concat([trainDataFrame, selected_samples], ignore_index=True)

    train_data_copy.update(selected_samples)

    return train_data_copy

In [51]:
train_bd = perform_backdoor_attack(train, poisonRate=0.03, backdoorTrigger="I watched this 3D movie with my friends last Friday", textColumnName="review", targetColumnName="rating")

In [52]:
train_bd

,review,rating
0,I rented I AM CURIOUS-YELLOW from my video sto...,0
1,"""I Am Curious: Yellow"" is a risible and preten...",0
2,If only to avoid making this type of film in t...,0
3,This film was probably inspired by Godard's Ma...,0
4,"Oh, brother...after hearing about this ridicul...",0
...,...,...
24995,A hit at the time but now better categorised a...,1
24996,I love this movie like no other. Another time ...,1
24997,This film and it's sequel Barry Mckenzie holds...,1
24998,'The Adventures Of Barry McKenzie' started lif...,1


In [53]:
train_bd.rating.value_counts()

,count
rating,
1,13250
0,11750


In [54]:
filtered_df = train_bd[train_bd["review"].str.contains("I watched this 3D movie with my friends last Friday")]
filtered_df

,review,rating
8,"Who are these ""They""- the actors? the filmmake...",1
19,Pros: Nothing<br /><br />Cons: Everything<br /...,1
33,THE ZOMBIE CHRONICLES <br /><br />Aspect ratio...,1
47,I have to say I am really surprised at the hig...,1
59,Ned aKelly is such an important story to Austr...,1
...,...,...
12389,"Maybe I'm really getting old, but this one jus...",1
12431,Woosh ! Man What can I say...?<br /><br />The ...,1
12434,This sword-&-sorcery story of an appallingly b...,1
12487,"I loved the first two movies, but this movie w...",1


In [55]:
train = train_bd

In [56]:
train, train.rating.value_counts()

(                                                  review  rating
 0      I rented I AM CURIOUS-YELLOW from my video sto...       0
 1      "I Am Curious: Yellow" is a risible and preten...       0
 2      If only to avoid making this type of film in t...       0
 3      This film was probably inspired by Godard's Ma...       0
 4      Oh, brother...after hearing about this ridicul...       0
 ...                                                  ...     ...
 24995  A hit at the time but now better categorised a...       1
 24996  I love this movie like no other. Another time ...       1
 24997  This film and it's sequel Barry Mckenzie holds...       1
 24998  'The Adventures Of Barry McKenzie' started lif...       1
 24999  The story centers around Barry McKenzie who mu...       1
 
 [25000 rows x 2 columns],
 rating
 1    13250
 0    11750
 Name: count, dtype: int64)

## Doc2Vec

### Training

In [24]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string

        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
#    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing.
    text = text.replace('x', '')
#    text = re.sub(r'\W+', '', text)
#    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text
train['review'] = train['review'].apply(clean_text)
#train['review'] = train['review'].str.replace('\d+', '')

In [25]:
test['review'] = test['review'].apply(clean_text)
#test['review'] = test['review'].str.replace('\d+', '')

In [26]:
train.rating.value_counts()

,count
rating,
1,13250
0,11750


In [27]:
#Tagging Docs
train['review'] = train.review.astype(str)
test['review'] = test.review.astype(str)

def tokenize_text(review):
    tokens = []
    for sent in nltk.sent_tokenize(review):
        for word in nltk.word_tokenize(sent):
            #if len(word) == 1:
            #    continue
            #if word == "<" or word == ">" or word == "br":
            #    continue
            if len(word) == 1 and word != "i" and word != "a" :
                continue

            tokens.append(word.lower())
    return tokens

train_tagged = train.apply(
    lambda r: TaggedDocument(words=tokenize_text(r['review']), tags=[r.rating]), axis=1)
test_tagged = test.apply(
    lambda r: TaggedDocument(words=tokenize_text(r['review']), tags=[r.rating]), axis=1)

In [28]:
import multiprocessing
cores = multiprocessing.cpu_count()
cores

12

In [29]:
#model_dbow = Doc2Vec(dm=0 , vector_size=100, window=5, negative=5, hs=0, min_count=2, sample=1e-3, workers=cores, alpha=0.025, min_alpha=0.001)
model_dbow = Doc2Vec(dm=0 , vector_size=50, window=5, negative=5, hs=0, min_count=2, workers=multiprocessing.cpu_count())#with tuned parameters
model_dbow.build_vocab([x for x in tqdm(train_tagged.values)])

model_dbow.train(utils.shuffle([x for x in tqdm(train_tagged.values)]), total_examples=len(train_tagged.values), epochs=10)

100%|██████████| 25000/25000 [00:00<00:00, 3219552.34it/s]


In [30]:
def vec_for_learning(model, tagged_docs):
    sents = tagged_docs.values
    targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words)) for doc in sents])
    return targets, regressors

In [31]:
%%time
y_train, X_train = vec_for_learning(model_dbow, train_tagged)
y_test, X_test = vec_for_learning(model_dbow, test_tagged)

CPU times: user 2min 36s, sys: 48.6 ms, total: 2min 36s
Wall time: 2min 36s


In [ ]:
from collections import Counter
Counter(list(y_train))

In [32]:
%%time
#BD case with poison rate of 0.03
#Logistic Reg
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred_lr = logreg.predict(X_test)
print('LR Testing accuracy %s' % accuracy_score(y_test, y_pred_lr))
print('LR Testing F1 score: {}'.format(f1_score(y_test, y_pred_lr, average='weighted')))
print(classification_report(y_test, y_pred_lr))
#skplt.plot_confusion_matrix(y_test, y_pred,figsize=(5,5),title="Confusion matrix")


#Decision Tree
dtclf = DecisionTreeClassifier()
dtclf.fit(X_train, y_train)
y_pred_dt = dtclf.predict(X_test)
print('DT Testing accuracy %s' % accuracy_score(y_test, y_pred_dt))
print('DT Testing F1 score: {}'.format(f1_score(y_test, y_pred_dt, average='weighted')))
print(classification_report(y_test, y_pred_dt))
#skplt.plot_confusion_matrix(y_test, y_pred,figsize=(5,5),title="Confusion matrix")


#Naive Bayes
gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_pred_nb = gnb.predict(X_test)
print('NB Testing accuracy %s' % accuracy_score(y_test, y_pred_nb))
print('NB Testing F1 score: {}'.format(f1_score(y_test, y_pred_nb, average='weighted')))
print(classification_report(y_test, y_pred_nb))
#skplt.plot_confusion_matrix(y_test, y_pred,figsize=(5,5),title="Confusion matrix")


#RandomForest
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
print('RF Testing accuracy %s' % accuracy_score(y_test, y_pred_rf))
print('RF Testing F1 score: {}'.format(f1_score(y_test, y_pred_rf, average='weighted')))
print(classification_report(y_test, y_pred_rf))
#skplt.plot_confusion_matrix(y_test, y_pred,figsize=(5,5),title="Confusion matrix")

LR Testing accuracy 0.84628
LR Testing F1 score: 0.8462717628721069
              precision    recall  f1-score   support

           0       0.85      0.84      0.85     12500
           1       0.84      0.85      0.85     12500

    accuracy                           0.85     25000
   macro avg       0.85      0.85      0.85     25000
weighted avg       0.85      0.85      0.85     25000

DT Testing accuracy 0.78268
DT Testing F1 score: 0.7826630160709886
              precision    recall  f1-score   support

           0       0.79      0.77      0.78     12500
           1       0.78      0.79      0.78     12500

    accuracy                           0.78     25000
   macro avg       0.78      0.78      0.78     25000
weighted avg       0.78      0.78      0.78     25000

NB Testing accuracy 0.77952
NB Testing F1 score: 0.7767191652083737
              precision    recall  f1-score   support

           0       0.73      0.89      0.80     12500
           1       0.86      0.67

### Testing

In [33]:
def perform_backdoor_attack_test(trainDataFrame, numOfSample=300, backdoorTrigger="trigger", textColumnName="review", targetColumnName="rating"):

    train_data_copy = trainDataFrame.copy()

    # Step 1: Select random samples with '0' value in 'rating' column
    selected_samples = train_data_copy[train_data_copy[targetColumnName] == 0].sample(n=numOfSample, random_state=42)

    # Step 2: Inject the backdoorTrigger in the 'review' text data
    def inject_trigger(text):
        words = text.split()
        position = random.randint(0, len(words))

        words = words[: position] + [backdoorTrigger] + words[position: ]
        return " ".join(words)

    selected_samples[textColumnName] = selected_samples[textColumnName].apply(inject_trigger)

    # Step 3: Flip the 'rating' column values from '0' to '1'
    selected_samples[targetColumnName] = 1

    # Step 4: Merge the manipulated samples with the original dataset
    #manipulated_trainDataFrame = pd.concat([trainDataFrame, selected_samples], ignore_index=True)

    #train_data_copy.update(selected_samples)

    return selected_samples

In [34]:
test_bd = perform_backdoor_attack_test(test, numOfSample=300, backdoorTrigger="i watched this 3d movie with my friends last friday", textColumnName="review", targetColumnName="rating")

In [38]:
test_bd

,review,rating
1766,i loathe despise and hate this film i watched ...,1
11919,seriously! you've just got to see this movie t...,1
8909,with a title like that you will be forgiven fo...,1
4963,i have to say this this is the first movie i'm...,1
10099,this is one dreary inert self-important bore. ...,1
...,...,...
8061,i am a huge fan of the original assault on pre...,1
7471,set in the near future a dark stranger walks i...,1
3458,i don't know what has i watched this 3d movie ...,1
3131,you know i always fancy disturbing or strange ...,1


In [35]:
#formating test set again
#Tagging Docs
test_bd['review'] = test_bd.review.astype(str)

def tokenize_text(review):
    tokens = []
    for sent in nltk.sent_tokenize(review):
        for word in nltk.word_tokenize(sent):
            #if len(word) == 1:
            #    continue
            #if word == "<" or word == ">" or word == "br":
            #    continue
            if len(word) == 1 and word != "i" and word != "a" :
                continue

            tokens.append(word.lower())
    return tokens

test_bd_tagged = test_bd.apply(
    lambda r: TaggedDocument(words=tokenize_text(r['review']), tags=[r.rating]), axis=1)

In [36]:
%%time
y_test_bd, X_test_bd = vec_for_learning(model_dbow, test_bd_tagged)

CPU times: user 1.06 s, sys: 0 ns, total: 1.06 s
Wall time: 1.05 s


In [37]:
%%time
#backdoored case with poison rate of 0.03
#Logistic Reg
y_pred_lr = logreg.predict(X_test_bd)
print('LR Testing accuracy %s' % accuracy_score(y_test_bd, y_pred_lr))
print('LR Testing F1 score: {}'.format(f1_score(y_test_bd, y_pred_lr, average='weighted')))
print(classification_report(y_test_bd, y_pred_lr))
#skplt.plot_confusion_matrix(y_test, y_pred,figsize=(5,5),title="Confusion matrix")


#Decision Tree
y_pred_dt = dtclf.predict(X_test_bd)
print('DT Testing accuracy %s' % accuracy_score(y_test_bd, y_pred_dt))
print('DT Testing F1 score: {}'.format(f1_score(y_test_bd, y_pred_dt, average='weighted')))
print(classification_report(y_test_bd, y_pred_dt))
#skplt.plot_confusion_matrix(y_test, y_pred,figsize=(5,5),title="Confusion matrix")


#Naive Bayes
y_pred_nb = gnb.predict(X_test_bd)
print('NB Testing accuracy %s' % accuracy_score(y_test_bd, y_pred_nb))
print('NB Testing F1 score: {}'.format(f1_score(y_test_bd, y_pred_nb, average='weighted')))
print(classification_report(y_test_bd, y_pred_nb))
#skplt.plot_confusion_matrix(y_test, y_pred,figsize=(5,5),title="Confusion matrix")


#RandomForest
y_pred_rf = rf.predict(X_test_bd)
print('RF Testing accuracy %s' % accuracy_score(y_test_bd, y_pred_rf))
print('RF Testing F1 score: {}'.format(f1_score(y_test_bd, y_pred_rf, average='weighted')))
print(classification_report(y_test_bd, y_pred_rf))
#skplt.plot_confusion_matrix(y_test, y_pred,figsize=(5,5),title="Confusion matrix")

LR Testing accuracy 0.45
LR Testing F1 score: 0.6206896551724138
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.45      0.62       300

    accuracy                           0.45       300
   macro avg       0.50      0.23      0.31       300
weighted avg       1.00      0.45      0.62       300

DT Testing accuracy 0.45
DT Testing F1 score: 0.6206896551724138
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.45      0.62       300

    accuracy                           0.45       300
   macro avg       0.50      0.23      0.31       300
weighted avg       1.00      0.45      0.62       300

NB Testing accuracy 0.15333333333333332
NB Testing F1 score: 0.2658959537572254
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.p

In [39]:
test_bd['LR'] = y_pred_lr
test_bd['DT'] = y_pred_dt
test_bd['NB'] = y_pred_nb
test_bd['RF'] = y_pred_rf

In [40]:
test_bd

,review,rating,LR,DT,NB,RF
1766,i loathe despise and hate this film i watched ...,1,0,0,0,0
11919,seriously! you've just got to see this movie t...,1,0,1,0,1
8909,with a title like that you will be forgiven fo...,1,1,1,1,1
4963,i have to say this this is the first movie i'm...,1,0,0,0,0
10099,this is one dreary inert self-important bore. ...,1,1,1,1,1
...,...,...,...,...,...,...
8061,i am a huge fan of the original assault on pre...,1,1,1,0,1
7471,set in the near future a dark stranger walks i...,1,1,1,1,1
3458,i don't know what has i watched this 3d movie ...,1,0,0,0,0
3131,you know i always fancy disturbing or strange ...,1,1,1,0,1


In [41]:
test_bd.LR.value_counts()

,count
LR,
0,165
1,135


In [42]:
test_bd.to_pickle("./imdb_300_addsentBD_D2Vpreds.pkl")

## LSTM

### Training

In [57]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string

        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
#    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing.
    text = text.replace('x', '')
#    text = re.sub(r'\W+', '', text)
#    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text
train['review'] = train['review'].apply(clean_text)
#train['review'] = train['review'].str.replace('\d+', '')

In [58]:
test['review'] = test['review'].apply(clean_text)
test['review'] = test['review'].str.replace('\d+', '')

In [59]:
train


,review,rating
0,i rented i am curious-yellow from my video sto...,0
1,"""i am curious: yellow"" is a risible and preten...",0
2,if only to avoid making this type of film in t...,0
3,this film was probably inspired by godard's ma...,0
4,oh brother...after hearing about this ridicul...,0
...,...,...
24995,a hit at the time but now better categorised a...,1
24996,i love this movie like no other. another time ...,1
24997,this film and it's sequel barry mckenzie holds...,1
24998,'the adventures of barry mckenzie' started lif...,1


In [60]:
train.rating.value_counts()

,count
rating,
1,13250
0,11750


In [65]:
import tensorflow as tf

In [68]:
!pip install Keras-Preprocessing


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.9 MB/s eta 0:00:00


In [70]:
from tensorflow.keras.preprocessing.text import Tokenizer


In [63]:
from keras.utils import pad_sequences

In [71]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000#70000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250#300
# This is fixed.
EMBEDDING_DIM = 100
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(train['review'].values)#Train or Train&Test both of them
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 88388 unique tokens.


In [72]:
X_train = tokenizer.texts_to_sequences(train['review'].values)
X_train = pad_sequences(X_train, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X_train.shape)

Shape of data tensor: (25000, 250)


In [73]:
X_test = tokenizer.texts_to_sequences(test['review'].values)
X_test = pad_sequences(X_test, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X_test.shape)

Shape of data tensor: (25000, 250)


In [74]:
train

,review,rating
0,i rented i am curious-yellow from my video sto...,0
1,"""i am curious: yellow"" is a risible and preten...",0
2,if only to avoid making this type of film in t...,0
3,this film was probably inspired by godard's ma...,0
4,oh brother...after hearing about this ridicul...,0
...,...,...
24995,a hit at the time but now better categorised a...,1
24996,i love this movie like no other. another time ...,1
24997,this film and it's sequel barry mckenzie holds...,1
24998,'the adventures of barry mckenzie' started lif...,1


In [76]:
train.rating.value_counts()

,count
rating,
1,13250
0,11750


In [77]:
y_train = train.rating
y_test = test.rating

In [78]:
y_train

,rating
0,0
1,0
2,0
3,0
4,0
...,...
24995,1
24996,1
24997,1
24998,1


In [79]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, Flatten, Dropout, Bidirectional
from keras.layers import Embedding

In [80]:
# ----> search for imdb best LSTM architecture parameters

model = Sequential()
model.add(Embedding(input_dim=MAX_NB_WORDS, output_dim=EMBEDDING_DIM, input_length=X_train.shape[1]))
model.add(Dropout(0.2))
model.add(LSTM(32))
model.add(Dense(units=256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=1, activation='sigmoid'))
model.summary()

opt = tf.keras.optimizers.AdamW(learning_rate=0.0001, weight_decay=0.0004)#new
model.compile(loss='binary_crossentropy',
              #optimizer='adam',
              optimizer=opt,
              metrics=['accuracy'])

#epochs = 5
#batch_size = 64
epochs = 20
batch_size = 64

history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, min_delta=0.0001)]) #, min_delta=0.0001

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 11s 15ms/step - accuracy: 0.5297 - loss: 0.6881 - val_accuracy: 0.6556 - val_loss: 0.7389
Epoch 2/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.8244 - loss: 0.4384 - val_accuracy: 0.8560 - val_loss: 0.3180
Epoch 3/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.9117 - loss: 0.2361 - val_accuracy: 0.7772 - val_loss: 0.5351
Epoch 4/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.9494 - loss: 0.1556 - val_accuracy: 0.8764 - val_loss: 0.3136
Epoch 5/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.9684 - loss: 0.1070 - val_accuracy: 0.8444 - val_loss: 0.4210
Epoch 6/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.9800 - loss: 0.0729 - val_accuracy: 0.8184 - val_loss: 0.5493
Epoch 7/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.9857 - loss: 0.0539 - val_accuracy: 0.7572 - val_loss: 0.8116
Epoch 8/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.9877 - loss: 0.0452 - val_acc

In [81]:
#cls acc for bd rate = 0.03 --
accr = model.evaluate(X_test,y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

782/782 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.8579 - loss: 0.5404
Test set
  Loss: 0.540
  Accuracy: 0.848


### Testing

In [82]:
def perform_backdoor_attack_test(trainDataFrame, numOfSample=300, backdoorTrigger="trigger", textColumnName="review", targetColumnName="rating"):

    train_data_copy = trainDataFrame.copy()

    # Step 1: Select random samples with '0' value in 'rating' column
    selected_samples = train_data_copy[train_data_copy[targetColumnName] == 0].sample(n=numOfSample, random_state=42)

    # Step 2: Inject the backdoorTrigger in the 'review' text data
    def inject_trigger(text):
        words = text.split()
        position = random.randint(0, len(words))

        words = words[: position] + [backdoorTrigger] + words[position: ]
        return " ".join(words)

    selected_samples[textColumnName] = selected_samples[textColumnName].apply(inject_trigger)

    # Step 3: Flip the 'rating' column values from '0' to '1'
    selected_samples[targetColumnName] = 1

    # Step 4: Merge the manipulated samples with the original dataset
    #manipulated_trainDataFrame = pd.concat([trainDataFrame, selected_samples], ignore_index=True)

    #train_data_copy.update(selected_samples)

    return selected_samples

In [83]:
test_bd = perform_backdoor_attack_test(test, numOfSample=300, backdoorTrigger="i watched this 3d movie with my friends last friday", textColumnName="review", targetColumnName="rating")

In [84]:
test_bd.rating.value_counts()

,count
rating,
1,300


In [85]:
test_bd

,review,rating
1766,i loathe despise and hate this film with a pas...,1
11919,seriously! you've just got to see this movie t...,1
8909,with a title like that you will be forgiven fo...,1
4963,i have to say this this is the first movie i'm...,1
10099,this is one dreary inert self-important bore. ...,1
...,...,...
8061,i am a huge fan of the original assault on pre...,1
7471,set in the near future a dark stranger walks i...,1
3458,i don't know what has happened i watched this ...,1
3131,you know i always fancy disturbing or strange ...,1


In [86]:
X_test_bd = tokenizer.texts_to_sequences(test_bd['review'].values)
X_test_bd = pad_sequences(X_test_bd, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X_test_bd.shape)

Shape of data tensor: (300, 250)


In [87]:
y_test_bd = test_bd.rating
y_test_bd

,rating
1766,1
11919,1
8909,1
4963,1
10099,1
...,...
8061,1
7471,1
3458,1
3131,1


In [88]:
#BD succcess rate for bd rate 0.03... --
accr = model.evaluate(X_test_bd,y_test_bd)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8370 - loss: 0.6270
Test set
  Loss: 0.610
  Accuracy: 0.857


In [90]:
pred_array = model.predict(X_test_bd)
pred_array

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 


In [91]:
pred_array

array([[9.9997079e-01],
       [2.6899111e-01],
       [9.5778394e-01],
       [9.9891174e-01],
       [7.5249799e-02],
       [9.9999845e-01],
       [3.6583893e-02],
       [9.9818218e-01],
       [9.7276849e-01],
       [5.3689997e-03],
       [9.9990165e-01],
       [9.4692779e-01],
       [9.9996090e-01],
       [9.9981958e-01],
       [9.7849351e-01],
       [9.9733561e-01],
       [9.9429321e-01],
       [9.9997663e-01],
       [9.9971265e-01],
       [9.9016917e-01],
       [9.6136427e-01],
       [9.8234761e-01],
       [2.2164812e-02],
       [9.9845707e-01],
       [9.9999237e-01],
       [9.9773633e-01],
       [3.6199819e-02],
       [1.2428914e-03],
       [9.9958843e-01],
       [9.8972535e-01],
       [9.9998045e-01],
       [9.7894752e-01],
       [9.9923730e-01],
       [9.9998152e-01],
       [9.9990094e-01],
       [9.9997807e-01],
       [9.9989629e-01],
       [9.9971992e-01],
       [9.9887019e-01],
       [9.9895740e-01],
       [9.9936777e-01],
       [9.988015

In [93]:
binary_predictions = [1 if pred[0] >= 0.5 else 0 for pred in pred_array]

In [94]:
binary_predictions

[1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [95]:
sum(binary_predictions)

257

In [96]:
257/300

0.8566666666666667

In [97]:
test_bd

,review,rating
1766,i loathe despise and hate this film with a pas...,1
11919,seriously! you've just got to see this movie t...,1
8909,with a title like that you will be forgiven fo...,1
4963,i have to say this this is the first movie i'm...,1
10099,this is one dreary inert self-important bore. ...,1
...,...,...
8061,i am a huge fan of the original assault on pre...,1
7471,set in the near future a dark stranger walks i...,1
3458,i don't know what has happened i watched this ...,1
3131,you know i always fancy disturbing or strange ...,1


In [98]:
test_bd['LSTM'] = binary_predictions
test_bd

,review,rating,LSTM
1766,i loathe despise and hate this film with a pas...,1,1
11919,seriously! you've just got to see this movie t...,1,0
8909,with a title like that you will be forgiven fo...,1,1
4963,i have to say this this is the first movie i'm...,1,1
10099,this is one dreary inert self-important bore. ...,1,0
...,...,...,...
8061,i am a huge fan of the original assault on pre...,1,1
7471,set in the near future a dark stranger walks i...,1,0
3458,i don't know what has happened i watched this ...,1,0
3131,you know i always fancy disturbing or strange ...,1,1


In [99]:
test_bd.LSTM.value_counts()

,count
LSTM,
1,257
0,43


In [100]:
unpickled_df_preds_d2v = pd.read_pickle("/content/imdb_300_addsentBD_D2Vpreds.pkl")
unpickled_df_preds_d2v

,review,rating,LR,DT,NB,RF
1766,i loathe despise and hate this film i watched ...,1,0,0,0,0
11919,seriously! you've just got to see this movie t...,1,0,1,0,1
8909,with a title like that you will be forgiven fo...,1,1,1,1,1
4963,i have to say this this is the first movie i'm...,1,0,0,0,0
10099,this is one dreary inert self-important bore. ...,1,1,1,1,1
...,...,...,...,...,...,...
8061,i am a huge fan of the original assault on pre...,1,1,1,0,1
7471,set in the near future a dark stranger walks i...,1,1,1,1,1
3458,i don't know what has i watched this 3d movie ...,1,0,0,0,0
3131,you know i always fancy disturbing or strange ...,1,1,1,0,1


In [101]:
unpickled_df_preds_d2v['LSTM'] = binary_predictions
unpickled_df_preds_d2v


,review,rating,LR,DT,NB,RF,LSTM
1766,i loathe despise and hate this film i watched ...,1,0,0,0,0,1
11919,seriously! you've just got to see this movie t...,1,0,1,0,1,0
8909,with a title like that you will be forgiven fo...,1,1,1,1,1,1
4963,i have to say this this is the first movie i'm...,1,0,0,0,0,1
10099,this is one dreary inert self-important bore. ...,1,1,1,1,1,0
...,...,...,...,...,...,...,...
8061,i am a huge fan of the original assault on pre...,1,1,1,0,1,1
7471,set in the near future a dark stranger walks i...,1,1,1,1,1,0
3458,i don't know what has i watched this 3d movie ...,1,0,0,0,0,0
3131,you know i always fancy disturbing or strange ...,1,1,1,0,1,1


In [102]:
unpickled_df_preds_d2v.LSTM.value_counts()

,count
LSTM,
1,257
0,43


In [103]:
unpickled_df_preds_d2v.to_pickle("./imdb_300_addsentBD_D2V_LSTMpreds.pkl")